In [0]:
STYLE_IMG_PATH = "content/Tuebingen_Neckarfront.jpg"
CONTENT_IMG_PATH = "content/Vassily_Kandinsky,_1913_-_Composition_7.jpg"

In [0]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools
import IPython.display

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models 
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

In [0]:
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [0]:
# Set up some global values here
style_path = "/" + STYLE_IMG_PATH
content_path = "/" + CONTENT_IMG_PATH

## Visualize the input

In [0]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [0]:
def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

These are input content and style images. We hope to "create" an image with the content of our content image, but with the style of the style image. 

In [0]:
plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')
print(content.shape)
plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()

In [0]:
def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

In [0]:
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [0]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

In [0]:
def get_model(content_layers=content_layers, style_layers=style_layers):
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    content_outputs = [vgg.get_layer(name).output for name in content_layers]
    style_outputs = [vgg.get_layer(name).output for name in style_layers]
    # Get output layers corresponding to style and content layers
    model_outputs = content_outputs + style_outputs
    # Build model 
    return models.Model(vgg.input, model_outputs)

In [0]:
model = get_model()
content_processed = load_and_process_img(content_path)

In [0]:
ENABLE_TESTING = False

In [0]:
def get_gram_matrix(input_tensor):
    num_filters = input_tensor.shape[-1] 
    reshaped_tensor = tf.reshape(input_tensor, (-1, num_filters))
    
    gram_matrix = tf.matmul(reshaped_tensor, reshaped_tensor, transpose_a=True)
    return gram_matrix

def get_content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - target))

def get_style_loss(base_style, gram_target):
    _, height, width, channels = base_style.get_shape().as_list()
    gram_style = get_gram_matrix(base_style)
    norm_term = tfe.Variable( 1.0 / (4. * (channels ** 2) * (width * height) ** 2), dtype='float32')
    return tf.multiply(tf.reduce_sum(tf.square(gram_style - gram_target)), norm_term)

        
if ENABLE_TESTING:
    x = np.array([1,1])
    y = np.array([0,10])
    with tf.GradientTape() as tape:
        print("content loss: ", get_content_loss(x,y))
    
    # gram
    tensor = np.array([[1, 1, 1], [2, 2, 2]], dtype='float32')
    tensor = tf.reshape(tensor, [1,3,1,2])
    print("reshaped tensor: ", tensor)
    gram_matrix = get_gram_matrix(tensor)
    print("gram matrix: ", gram_matrix)
    
    #style loss
    base_style = tensor
    with tf.GradientTape() as tape:
        target_gram = get_gram_matrix(tensor)
        style_loss = get_style_loss(base_style, target_gram)
        print("style loss: ", style_loss)
    

In [0]:
def get_feature_representations(model, content_path, style_path, num_style_layers):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """
  # Load our images in 
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)
  
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

In [0]:
def get_total_loss(model, num_style_layers, num_content_layers,
                   loss_weights, init_image, gram_targets, content_targets):
    
    
    content_weight, style_weight = loss_weights
    
    model_outputs = model(init_image)

    style_output_features = model_outputs[:num_style_layers]
    content_output_features = model_outputs[num_style_layers:]
    
    weight_per_style_layer = 1.0 / float(num_style_layers)
    weight_per_content_layer = 1.0 / float(num_content_layers)
    style_score, content_score = 0, 0
    
    for gram_target, style_feature in zip(gram_targets, style_output_features):
        style_score = weight_per_style_layer * get_style_loss(style_feature, gram_target)
        
    for content_target, content_feature in zip(content_targets, content_output_features):
        content_score += weight_per_content_layer * get_content_loss(content_feature, content_target)

    content_loss = content_weight * content_score 
    style_loss = style_weight * style_score
    
    total_loss = content_loss + style_loss
    return total_loss, content_loss, style_loss

def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = get_total_loss(**cfg)
        
    total_loss = all_loss[0]
    return tape.gradient(total_loss, cfg['init_image']), all_loss

In [0]:
def run_style_transfer(content_path, style_path, num_iterations, content_layers, 
                       style_layers, init_image=None, 
                       style_weight=10e-2, content_weight=1e3):
    ''' returns a list of tuples of images and their losses
    obtained during style transfer'''
        
    model = get_model()
    
    for layer in model.layers:
        layer.trainable = False
    
    content_img = load_and_process_img(content_path)
    style_img = load_and_process_img(style_path)
    
    if init_image == None:
        init_image = content_img
    init_image = tfe.Variable(init_image)
    
    style_features, content_features = get_feature_representations(model, content_path, style_path, len(style_layers)) # TO DO
    print(style_features[0].shape)
    gram_targets = [get_gram_matrix(style_feat) for style_feat in style_features]
    content_targets = content_features
    
    # Create our optimizer
    opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)
    
    loss_weights = (content_weight, style_weight)
    cfg = {
        'model': model,
        'num_style_layers': len(style_layers),
        'num_content_layers': len(content_layers),
        'loss_weights': loss_weights,
        'init_image': init_image, # not sure if it will work 
        'gram_targets': gram_targets,
        'content_targets': content_targets
    }
    
    # For displaying intermediate images 
    iter_count = 1

    # Store our best result
    best_loss, best_img = float('inf'), None
    
    num_rows = 2
    num_cols = 5
    display_interval = int(num_iterations/(num_rows*num_cols))
    start_time = time.time()
    global_start = time.time()

    norm_means = np.array([103.939, 116.779, 123.68])
    min_vals = -norm_means
    max_vals = 255 - norm_means   

    imgs = []

    for i in range(num_iterations):
      #print(i, display_interval)
      grads, all_loss = compute_grads(cfg) # TO DO
      loss, content_loss, style_loss = all_loss
      opt.apply_gradients([(grads, init_image)])
      clipped = tf.clip_by_value(init_image, min_vals, max_vals)
      init_image.assign(clipped)

      end_time = time.time() 

      if loss < best_loss:
        # Update best loss and best image from total loss. 
        best_loss = loss
        best_img = deprocess_img(init_image.numpy())

      if i % display_interval== 0:
        start_time = time.time()

        # Use the .numpy() method to get the concrete numpy array
        plot_img = init_image.numpy()
        plot_img = deprocess_img(plot_img)
        imgs.append(plot_img)
        IPython.display.clear_output(wait=True)
        IPython.display.display_png(Image.fromarray(plot_img))
        print('Iteration: {}'.format(i))        
        print('Total loss: {:.4e}, ' 
              'style loss: {:.4e}, '
              'content loss: {:.4e}, '
              'time: {:.4f}s'.format(loss, style_loss, content_loss, time.time() - start_time))
    
    print('Total time: {:.4f}s'.format(time.time() - global_start))
    IPython.display.clear_output(wait=True)
    plt.figure(figsize=(14,4))
    for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])

    return best_img, best_loss 
    
    
    
    
    

In [0]:
best_img, _ = run_style_transfer(content_path, style_path, 1000, content_layers, style_layers)

In [0]:
def show_results(best_img, content_path, style_path, show_large_final=True):
  plt.figure(figsize=(10, 5))
  content = load_img(content_path) 
  style = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style, 'Style Image')

  if show_large_final: 
    plt.figure(figsize=(10, 10))
    plt.imshow(best_img)
    plt.title('Output Image')
    plt.show()

In [0]:
show_results(best, content_path, style_path)